In [19]:
from openai import OpenAI
from g4f.client import Client
from db_utils import csv_read,query_database
import re
import pandas as pd

In [20]:
client = Client()

In [21]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f''''
        1. 你是一個資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現沒有的英文名稱。
        5. 請務必生成完整SQL語法。
        '''

    user_prompt = f'''
        資料庫結構、表名稱、欄位名稱請務必參考以下內容:{csv_string}
        千萬不能生成沒有的sql表名與欄名!
        
                '''
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        max_tokens=150,
        temperature=0.1
    )
    sql_response = response.choices[0].message.content
    print(f"回答:{sql_response}")
    return sql_response


模型可用系列:GPT、Gemini

In [22]:
def extract_sql_query(sql_response): 
    match = re.search(r'SELECT.*?;', sql_response,re.IGNORECASE | re.DOTALL) 
    if match: 
        return match.group() 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [23]:
def main():
    user_input ="查詢員工的薪資以及相對應的薪資等級"   #input("請輸入內容: ") 
    # 初次調用GPT判斷是否需要function calling 
    response_1 = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":"請判斷使用者輸入是否需要連線到資料庫查詢，若需要請回答YES即可!若不需要則正常對話"},
                                                {"role": "user", "content": user_input}])
    
    print('===== 初次調用GPT判斷 =====')
    if 'YES' in response_1.choices[0].message.content:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_query(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        result = query_database(sql_query)
        print(f'查詢結果: {result}')
        return("OK")

    else: 
        # 一般對話處理
        return(response_1.choices[0].message.content)
         


In [24]:
main()

===== 初次調用GPT判斷 =====
回答:```sql
SELECT emp.EMPNO, emp.ENAME, emp.SAL, salgrade.GRADE
FROM emp
JOIN salgrade ON emp.SAL BETWEEN salgrade.LOSAL AND salgrade.HISAL;
```
生成的SQL指令: SELECT emp.EMPNO, emp.ENAME, emp.SAL, salgrade.GRADE
FROM emp
JOIN salgrade ON emp.SAL BETWEEN salgrade.LOSAL AND salgrade.HISAL;
連線成功
查詢結果: [{"EMPNO": 7369, "ENAME": "SMITH", "SAL": 800.0, "GRADE": 1}, {"EMPNO": 7499, "ENAME": "ALLEN", "SAL": 1600.0, "GRADE": 3}, {"EMPNO": 7521, "ENAME": "WARD", "SAL": 1250.0, "GRADE": 2}, {"EMPNO": 7566, "ENAME": "JONES", "SAL": 2975.0, "GRADE": 4}, {"EMPNO": 7654, "ENAME": "MARTIN", "SAL": 1250.0, "GRADE": 2}, {"EMPNO": 7698, "ENAME": "BLAKE", "SAL": 2850.0, "GRADE": 4}, {"EMPNO": 7782, "ENAME": "CLARK", "SAL": 2450.0, "GRADE": 4}, {"EMPNO": 7788, "ENAME": "SCOTT", "SAL": 3000.0, "GRADE": 4}, {"EMPNO": 7839, "ENAME": "KING", "SAL": 5000.0, "GRADE": 5}, {"EMPNO": 7844, "ENAME": "TURNER", "SAL": 1500.0, "GRADE": 3}, {"EMPNO": 7876, "ENAME": "ADAMS", "SAL": 1100.0, "GRADE": 1}, {

'OK'

C:\Users\TMP214\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py:80: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


測試判斷是否連線財報資料庫

In [6]:
if __name__ == "__main__":
    user_input ="請幫我查詢112年第一季收入。"   #input("請輸入內容: ") 
    response_1 = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":'''1.請判斷使用者的輸入是否需要連線到財務報表資料庫查詢，若需要請回答YES即可!
                                                              2.若不用連線只要回答NO即可!
                                                              2.財務報表資料內容包含資產負債表、損益表、現金流量表，其餘公司資料就沒有。
                                                            
                                                            '''},
                                                {"role": "user", "content": user_input}])
    print(response_1.choices[0].message.content )

New g4f version: 0.3.9.1 (current: 0.3.8.1) | pip install -U g4f
YES


C:\Users\TMP214\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py:80: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


In [8]:
if __name__ == "__main__":
    user_input ="請幫我分析112年淨營收狀態。"   #input("請輸入內容: ") 
    response_1 = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":'''1.請判斷使用者的輸入是否需要連線到財務報表資料庫查詢，若需要請回答YES即可!
                                                              2.若不用連線只要回答NO即可!
                                                              2.財務報表資料內容包含資產負債表、損益表、現金流量表，其餘公司資料就沒有。
                                                            
                                                            '''},
                                                {"role": "user", "content": user_input}])
    print(response_1.choices[0].message.content )

YES
